# Parking Area Definition Tool

Simple tool to define parking areas for vehicle detection.

**Input**: Video file  
**Output**: JSON file with parking areas named `parking-areas-[video_name].json`

## How to use:
1. Set your video path in the cell below
2. Run the cells
3. Click 4 points to define each parking area
4. Press 's' to save when done

In [5]:
import cv2
import numpy as np
import json
import os
from datetime import datetime

class ParkingAreaDefiner:
    def __init__(self, video_path):
        self.video_path = video_path
        # Generate output filename based on video name
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        self.output_path = f'../output/parking-areas-{video_name}.json'
        self.parking_areas = []
        self.current_polygon = []
        print(f"Video: {video_path}")
        print(f"Output will be saved to: {self.output_path}")
        
    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            if len(self.current_polygon) < 4:
                self.current_polygon.append([x, y])
                print(f"Point {len(self.current_polygon)}: ({x}, {y})")
                
                if len(self.current_polygon) == 4:
                    self.parking_areas.append(np.array(self.current_polygon))
                    print(f"✅ Area {len(self.parking_areas)} completed")
                    self.current_polygon = []
                    
        elif event == cv2.EVENT_RBUTTONDOWN:
            if self.current_polygon:
                self.current_polygon = []
                print("❌ Current area cancelled")
                
    def define_areas(self):
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            print(f"❌ Error opening video: {self.video_path}")
            return False
        
        ret, frame = cap.read()
        if not ret:
            print("❌ Error reading frame")
            return False
        cap.release()
        
        print("\n🎯 Define parking areas:")
        print("• Left click 4 points to create area")
        print("• Right click to cancel current area")
        print("• Press 'r' to delete last area")
        print("• Press 's' to save and exit")
        print("• Press 'q' to quit without saving")
        
        cv2.namedWindow('Define Parking Areas', cv2.WINDOW_NORMAL)
        cv2.setMouseCallback('Define Parking Areas', self.mouse_callback)
        
        while True:
            display_frame = frame.copy()
            
            # Draw completed areas
            for i, area in enumerate(self.parking_areas):
                cv2.polylines(display_frame, [area], True, (0, 255, 0), 2)
                overlay = display_frame.copy()
                cv2.fillPoly(overlay, [area], (0, 255, 0))
                display_frame = cv2.addWeighted(display_frame, 0.8, overlay, 0.2, 0)
                
                center = np.mean(area, axis=0).astype(int)
                cv2.putText(display_frame, f'P{i+1}', tuple(center), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)
                cv2.putText(display_frame, f'P{i+1}', tuple(center), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            # Draw current points
            for i, point in enumerate(self.current_polygon):
                cv2.circle(display_frame, tuple(point), 6, (0, 0, 255), -1)
                cv2.putText(display_frame, str(i+1), (point[0]+10, point[1]-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            
            # Draw lines between current points
            if len(self.current_polygon) >= 2:
                for i in range(len(self.current_polygon) - 1):
                    cv2.line(display_frame, 
                            tuple(self.current_polygon[i]), 
                            tuple(self.current_polygon[i+1]), 
                            (0, 0, 255), 2)
            
            # Status text
            status = f"Areas: {len(self.parking_areas)} | Points: {len(self.current_polygon)}/4"
            cv2.putText(display_frame, status, (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 3)
            cv2.putText(display_frame, status, (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.imshow('Define Parking Areas', display_frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('r') and self.parking_areas:
                self.parking_areas.pop()
                print(f"🗑️ Deleted area. Now have {len(self.parking_areas)} areas")
            elif key == ord('s'):
                return self.save_areas()
            elif key == ord('q'):
                print("❌ Exiting without saving")
                break
                
        cv2.destroyAllWindows()
        return False
        
    def save_areas(self):
        if not self.parking_areas:
            print("❌ No areas to save")
            return False
            
        os.makedirs(os.path.dirname(self.output_path), exist_ok=True)
        
        data = {
            'video_path': self.video_path,
            'created_at': datetime.now().isoformat(),
            'num_areas': len(self.parking_areas),
            'parking_areas': [area.tolist() for area in self.parking_areas]
        }
        
        try:
            with open(self.output_path, 'w') as f:
                json.dump(data, f, indent=2)
            
            print(f"\n✅ Saved {len(self.parking_areas)} parking areas")
            print(f"📁 File: {self.output_path}")
            cv2.destroyAllWindows()
            return True
            
        except Exception as e:
            print(f"❌ Error saving: {e}")
            return False

In [6]:
# 🎬 SET YOUR VIDEO PATH HERE
video_path = '../dataset/video_1_cut50-00_01-00-00.mov'

# Create definer and start
definer = ParkingAreaDefiner(video_path)
success = definer.define_areas()

if success:
    print("\n🎉 Parking areas defined successfully!")
    print("Use this file in parking-detection.ipynb")
else:
    print("\n❌ No areas were saved")

Video: ../dataset/video_1_cut50-00_01-00-00.mov
Output will be saved to: ../output/parking-areas-video_1_cut50-00_01-00-00.json

🎯 Define parking areas:
• Left click 4 points to create area
• Right click to cancel current area
• Press 'r' to delete last area
• Press 's' to save and exit
• Press 'q' to quit without saving
Point 1: (19, 402)
Point 2: (467, 19)
Point 3: (371, 13)
Point 4: (11, 169)
✅ Area 1 completed
Point 1: (807, 63)
Point 2: (1265, 459)
Point 3: (1269, 196)
Point 4: (877, 26)
✅ Area 2 completed

✅ Saved 2 parking areas
📁 File: ../output/parking-areas-video_1_cut50-00_01-00-00.json

🎉 Parking areas defined successfully!
Use this file in parking-detection.ipynb
